# Notebook 1: MED-PC Extracting the Recording Data and Metadata

# Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
from datetime import datetime

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from medpc2excel.medpc_read import medpc_read
from moviepy.editor import *

In [3]:
# setting path
sys.path.append('../../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

# Getting the Metadata from all the MED-PC Recording Files

- Every MED-PC Recording file has Metadata about time, data, subject, group, experiment name, script used, and the MED-PC box number on the top of the recording file. We will first extract that information
    - Metadata can be background information or context about data(or files) that is often different from the actual data that is intended to be recorded.  
        For more information on Metadata: https://en.wikipedia.org/wiki/Metadata

In [5]:
cohort = "pilot_1"

In [6]:
# Getting all the file paths of the recording files(that happen to all end in `.txt`)
all_med_pc_file = glob.glob("./data/*.txt")

- The path of the directory that this notebook is in. This will be where relative paths will be based off of

In [7]:
current_working_directory = os.getcwd()

In [8]:
current_working_directory

'/root/projects/behavioral_dataframe_processing/results/2022_10_12_pilot_1_2_3_reward_training/pilot_1'

- All the other files in this directory. If you want to use a folder in here such as the `data` folder, you'd type `./data/{name_of_folder}` where you replace `{name_of_folder}` with the name of the folder without the `{}`. You will do this in the cells following the one below
    - The `./` means the path will reference the current directory that the command is being used from

In [9]:
os.listdir(current_working_directory)

['data',
 '03_calculating_port_entry_precision.ipynb',
 '.ipynb_checkpoints',
 'proc',
 '02_calculating_latencies.ipynb',
 '01_extracting_recording_data_and_metadata.ipynb']

## NOTE: If you are using your own data, the following path must be changed to the directory where your MED-PC recording files are located(if they are not in the specied folder). It is recommended to create a sub folder in the `data` folder(that is in the same folder as this notebook) for each group of session you want to process and put the recording files in there

- Use the cell below instead if you're using your own data. You must change the cell below by clicking on the cell and then pressing the `esc` key. Then press the `Y` button on your keyboard. If you want to switch it back do the same but press the `R` button instead. This switches it from a cell that is run as if it's code, or back to a cell where it's just treated as text
    - **NOTE: If you are using a different folder, then change the path in the cell below**
    - The asterisk is called a wild card which will tell the computer to find all files/folders that match this pattern. The `*` means that any pattern of any length can be replaced with it. The `**` tells the computer to look in all folders that are in the specified folder. For more information: https://linuxhint.com/bash_wildcard_tutorial/
    - For more information on finding the path of your folder that contains the recording files: https://www.computerhope.com/issues/ch001708.htm

In [10]:
all_med_pc_file = glob.glob("./data/*.txt")

In [11]:
all_med_pc_file[:10]

['./data/2022-05-16_11h28m_Subject 2.1.txt',
 './data/2022-05-04_10h11m_Subject 2.1.txt',
 './data/2022-05-06_12h59m_Subject 3.1 (1).txt',
 './data/2022-05-07_08h53m_Subject 1.4.txt',
 './data/2022-05-03_16h09m_Subject 3.4 (2).txt',
 './data/2022-05-04_13h06m_Subject 4.3 (3).txt',
 './data/2022-05-10_10h42m_Subject 2.4.txt',
 './data/2022-05-05_16h00m_Subject 3.1 (1) .txt',
 './data/2022-05-10_15h58m_Subject 1.1.txt',
 './data/2022-05-09_12h43m_Subject 4.1 (1).txt']

# Filtering the MED-PC Files that have Pilot of Pilot in it

- Example of what the MED-PC Recording file looks like

In [12]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-05-16_11h28m_Subject 2.1.txt







Start Date: 05/16/22

End Date: 05/16/22

Subject: 2.1

Experiment: Pilot of Pilot

Group: Cage 2

Box: 3

Start Time: 11:28:27

End Time: 12:35:20

MSN: levelNP_CS_reward_laserepochON1st_noshock

A:    5999.000

D:    9000.000

F:    2000.000

G:       0.000

H:       0.000

I:       0.000

L:       0.000



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```

- We will just find all the lines that start with `"File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN"`. And get the metadata from those lines. And then stop once all the metadata types have been collected

In [13]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [14]:
# The metadata for the first file
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/2022-05-16_11h28m_Subject 2.1.txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-05-16_11h28m_Subject 2.1.txt', 'Start Date': '05/16/22', 'End Date': '05/16/22', 'Subject': '2.1', 'Experiment': 'Pilot of Pilot', 'Group': 'Cage 2', 'Box': '3', 'Start Time': '11:28:27', 'End Time': '12:35:20', 'MSN': 'levelNP_CS_reward_laserepochON1st_noshock'}


## Making a Dataframe out of the Metadata

- A Dataframe is essentially a "programmable" spreadsheet. But instead of clicking on cells, you will have to tell Python how you want to interact with the spreadsheet
    - For more information: https://realpython.com/pandas-dataframe/

In [15]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [16]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/2022-05-16_11h28m_Subject 2.1.txt,C:\MED-PC\Data\2022-05-16_11h28m_Subject 2.1.txt,05/16/22,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock
1,./data/2022-05-04_10h11m_Subject 2.1.txt,C:\MED-PC\Data\2022-05-04_10h11m_Subject 2.1.txt,05/04/22,05/04/22,2.1,Pilot of Pilot,Cage 2,2,10:11:55,11:18:44,levelNP_CS_reward_laserepochON1st_noshock
2,./data/2022-05-06_12h59m_Subject 3.1 (1).txt,C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.1 (...,05/06/22,05/06/22,3.1 (1),Pilot of Pilot,Cage 4,4,12:59:58,14:02:38,levelNP_CS_reward_laserepochON1st_noshock
3,./data/2022-05-07_08h53m_Subject 1.4.txt,C:\MED-PC\Data\2022-05-07_08h53m_Subject 1.4.txt,05/07/22,05/07/22,1.4,Pilot of Pilot,Cage 1,3,08:53:46,10:21:47,levelNP_CS_reward_laserepochON1st_noshock
4,./data/2022-05-03_16h09m_Subject 3.4 (2).txt,C:\MED-PC\Data\2022-05-03_16h09m_Subject 3.4 (...,05/03/22,05/03/22,3.4 (2),Pilot of Pilot,Cage 4,2,16:09:34,17:19:39,levelNP_CS_reward_laserepochON1st_noshock


In [17]:
metadata_df.tail()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
188,./data/2022-05-07_12h20m_Subject 4.2 (3).txt,C:\MED-PC\Data\2022-05-07_12h20m_Subject 4.2 (...,05/07/22,05/07/22,4.2 (3),Pilot of Pilot,Cage 3,3,12:20:43,13:21:43,levelNP_CS_reward_laserepochON1st_noshock
189,./data/2022-05-03_12h52m_Subject 1.3.txt,C:\MED-PC\Data\2022-05-03_12h52m_Subject 1.3.txt,05/03/22,05/03/22,1.3,Pilot of Pilot,Cage 2,3,12:52:48,13:17:04,levelNP_CS_reward_laserepochON1st_noshock
190,./data/2022-05-07_12h20m_Subject 3.3 (4).txt,C:\MED-PC\Data\2022-05-07_12h20m_Subject 3.3 (...,05/07/22,05/07/22,3.3 (4),Pilot of Pilot,Cage 3,4,12:20:43,13:21:43,levelNP_CS_reward_laserepochON1st_noshock
191,./data/2022-05-11_10h27m_Subject 1.3.txt,C:\MED-PC\Data\2022-05-11_10h27m_Subject 1.3.txt,05/11/22,05/11/22,1.3,Pilot of Pilot,Cage 2,3,10:27:24,11:32:16,levelNP_CS_reward_laserepochON1st_noshock
192,./data/2022-05-09_12h43m_Subject 4.2 (3).txt,C:\MED-PC\Data\2022-05-09_12h43m_Subject 4.2 (...,05/09/22,05/09/22,4.2 (3),Pilot of Pilot,Cage 3,1,12:43:20,13:48:27,levelNP_CS_reward_laserepochON1st_noshock


- Getting the numbers out of the column that contains the cage information

In [18]:
import re

In [19]:
metadata_df = metadata_df.dropna(subset="Group")

In [20]:
metadata_df = metadata_df[metadata_df["Group"] != ""]

In [21]:
metadata_df["cage"] = metadata_df["Group"].apply(lambda x: re.findall(r'\d+', x)[0])

In [22]:
metadata_df["cage"].unique()

array(['2', '4', '1', '3'], dtype=object)

In [23]:
metadata_df["cage"].head()

0    2
1    2
2    4
3    1
4    4
Name: cage, dtype: object

- Getting the number of files that are associated with each subject

In [24]:
# How many files there are for each subject
metadata_df.groupby("Subject").count()

,index,File,Start Date,End Date,Experiment,Group,Box,Start Time,End Time,MSN,cage
Subject,,,,,,,,,,,
1.1,13,13,13,13,13,13,13,13,13,13,13
1.2,13,13,13,13,13,13,13,13,13,13,13
1.3,13,13,13,13,13,13,13,13,13,13,13
1.4,13,13,13,13,13,13,13,13,13,13,13
2.1,13,13,13,13,13,13,13,13,13,13,13
2.2,13,13,13,13,13,13,13,13,13,13,13
2.3,13,13,13,13,13,13,13,13,13,13,13
2.4,13,13,13,13,13,13,13,13,13,13,13
3.1 (1),11,11,11,11,11,11,11,11,11,11,11


## Looking over the MED-PC scripts

- MED-PC has scripts(lines of code that act as the instructions for how to operate the MED-PC boxes) that contain the descriptions of what each value in the recordings are. We will extract the descriptions from the MED-PC scripts so that we can label the data points in the MED-PC recordings  
- Below is a list of all the MED-PC Scripts. We will only be looking at the first one    

In [25]:
all_medpc_scripts = glob.glob("./data/*.MPC")

In [26]:
all_medpc_scripts

['./data/C57_reward_competition.MPC',
 './data/CD1_reward_training.MPC',
 './data/levelNP_CS_reward_laserepochON1st_noshock.MPC',
 './data/CD1_reward_competition.MPC',
 './data/CD1_reward.MPC',
 './data/C57_reward_training.MPC',
 './data/pumptest.MPC',
 './data/C57_reward.MPC',
 './data/port_entry_tester.MPC']

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [27]:
with open(all_medpc_scripts[0]) as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

\v3 stop tone with poke

\v3.2 monitor port entries AND exits



\INPUTS

^port = 8



\OUTPUTS

^fan = 16

^houselight = 11

^tone1 = 2

^tone2 = 3

^tone3 = 4

^tone4 = 5

^pump = 9

^whitenoise = 1

^csout = 5

^peout = 15

^cs1out = 17

^cs2out = 13

^cs3out = 14



\EXP SETTINGS

^ncsNoShock = 0

^initCS1trials = 3



\ARRAYS

DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM G = 2500 \controlled_stimulus_seconds computer clock time (seconds on clock every time light turns on)

DIM H = 2500 \controlled_stimulus_minutes computer clock time (minutes on clock every time light turns on)

DIM I = 2500 \controlled_stimulus_hours computer clock time (hours every time light turns on)

DIM B = 2500 \port

- We will be using the comments in the MED-PC script(Everything after the `\` for each line) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable(programming object that holds some information).
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the variable names and their comments in the MED-PC script that we will get the descriptive names from:
    - MED-PC will squish the first few words to create the name for every variable

```
DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM B = 2500 \shock intensity
```

- In the MED-PC recording files, there are values that are labelled with letters. These data type of these letters is described in the MED-PC script file that we just looked at.

In [28]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-05-16_11h28m_Subject 2.1.txt







Start Date: 05/16/22

End Date: 05/16/22

Subject: 2.1

Experiment: Pilot of Pilot

Group: Cage 2

Box: 3

Start Time: 11:28:27

End Time: 12:35:20

MSN: levelNP_CS_reward_laserepochON1st_noshock

A:    5999.000

D:    9000.000

F:    2000.000

G:       0.000

H:       0.000

I:       0.000

L:       0.000



## **NOTE: Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder as the recording files. This notebook will fail to extract the data from the recording file if it is missing**

- Spreadsheet of the data from the MED-PC recording file for all the files combined into one.
- **NOTE: Each row does not correspond to the same data point. Each row represents the "n"-th data point for each category for each file. AKA, the first row is the first data point for the time the subject entered the port and the first data point for the time that the tone that is played, two seperate things. The second row is the second data point for all the categories, the third row is the third data point and so on. This repeats for all data points in a given file, then it starts over to the next file where the first row of that file is the first for all its data points.**

In [29]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)

Traceback (most recent call last):
  File "/root/projects/behavioral_dataframe_processing/results/2022_10_12_pilot_1_2_3_reward_training/pilot_1/../../../src/extract/dataframe.py", line 71, in get_medpc_dataframe_from_list_of_files
    ts_df, medpc_log = medpc_read(file=file_path, override=True, replace=False)
  File "/root/projects/behavioral_dataframe_processing/behavioral_processing_env/lib/python3.9/site-packages/medpc2excel/medpc_read.py", line 114, in medpc_read
    temp += re.split('\s+',d.split(':')[1])
IndexError: list index out of range

Invalid Formatting for file: ./data/2022-05-03_11h42m_Subject 1.1.txt
Traceback (most recent call last):
  File "/root/projects/behavioral_dataframe_processing/results/2022_10_12_pilot_1_2_3_reward_training/pilot_1/../../../src/extract/dataframe.py", line 71, in get_medpc_dataframe_from_list_of_files
    ts_df, medpc_log = medpc_read(file=file_path, override=True, replace=False)
  File "/root/projects/behavioral_dataframe_processing/behaviora

In [30]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
0,3.95,64.0,399.0,0.0,60.01,5.03,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt
1,16.56,144.0,399.0,0.0,140.01,16.70,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt
2,16.85,234.0,399.0,0.0,230.01,16.88,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt
3,16.97,314.0,399.0,0.0,310.01,17.93,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt
4,24.95,389.0,399.0,0.0,385.01,25.28,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt


In [31]:
concatted_medpc_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
2536,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),./data/2022-05-09_12h43m_Subject 4.2 (3).txt
2537,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),./data/2022-05-09_12h43m_Subject 4.2 (3).txt
2538,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),./data/2022-05-09_12h43m_Subject 4.2 (3).txt
2539,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),./data/2022-05-09_12h43m_Subject 4.2 (3).txt
2540,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),./data/2022-05-09_12h43m_Subject 4.2 (3).txt


In [32]:
pilot_df = concatted_medpc_df[~concatted_medpc_df["subject"].str.contains(r'v', na=True)].copy()

In [33]:
pilot_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
0,3.95,64.0,399.0,0.0,60.01,5.03,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt
1,16.56,144.0,399.0,0.0,140.01,16.70,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt
2,16.85,234.0,399.0,0.0,230.01,16.88,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt
3,16.97,314.0,399.0,0.0,310.01,17.93,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt
4,24.95,389.0,399.0,0.0,385.01,25.28,1.0,0.0,20220516,2.1,./data/2022-05-16_11h28m_Subject 2.1.txt
...,...,...,...,...,...,...,...,...,...,...,...
2536,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),./data/2022-05-09_12h43m_Subject 4.2 (3).txt
2537,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),./data/2022-05-09_12h43m_Subject 4.2 (3).txt
2538,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),./data/2022-05-09_12h43m_Subject 4.2 (3).txt
2539,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),./data/2022-05-09_12h43m_Subject 4.2 (3).txt


In [34]:
pilot_df["date"].unique()

array(['20220516', '20220504', '20220506', '20220507', '20220503',
       '20220510', '20220505', '20220509', '20220508', '20220512',
       '20220511', '20220517'], dtype=object)

In [35]:
concatted_medpc_df = pilot_df.copy()

- Combining the recording and the metadata into one dataframe

In [36]:
recording_and_metadata_df = concatted_medpc_df.merge(metadata_df, left_on='file_path', right_on='index')

In [37]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
0,3.95,64.0,399.0,0.0,60.01,5.03,1.0,0.0,20220516,2.1,...,05/16/22,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2
1,16.56,144.0,399.0,0.0,140.01,16.70,1.0,0.0,20220516,2.1,...,05/16/22,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2
2,16.85,234.0,399.0,0.0,230.01,16.88,1.0,0.0,20220516,2.1,...,05/16/22,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2
3,16.97,314.0,399.0,0.0,310.01,17.93,1.0,0.0,20220516,2.1,...,05/16/22,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2
4,24.95,389.0,399.0,0.0,385.01,25.28,1.0,0.0,20220516,2.1,...,05/16/22,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2


## Getting the cage numbers and the dates so that we can include it in the file name

- We will get the metadata from the recording files and use it to name the files we will create. This will help organize the files and make it easy to know where each file came from 

- Getting the group numbers(original cage names)

In [38]:
# removing blank spaces
group_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Group"].unique() if number]
# sorting numbers
group_numbers = sorted(group_numbers)
group_numbers_for_title = "_".join(group_numbers)

In [39]:
group_numbers_for_title

'Cage1_Cage_1_Cage_2_Cage_3_Cage_4'

- Getting the cage numbers

In [40]:
# removing blank spaces
cage_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["cage"].unique() if number]
# sorting numbers
cage_numbers = sorted(cage_numbers)
cage_numbers_for_title = "_".join(cage_numbers)

In [41]:
cage_numbers_for_title

'1_2_3_4'

- Getting the dates

In [42]:
# Getting the first and last recording date to get a range
earliest_date = recording_and_metadata_df["date"].min()
latest_date = recording_and_metadata_df["date"].max()

In [43]:
earliest_date

'20220503'

In [44]:
latest_date

'20220517'

- Getting the subject names

In [45]:
recording_and_metadata_df["Subject"].unique()

array(['2.1', '3.1 (1)', '1.4', '3.4 (2)', '4.3 (3)', '2.4', '1.1',
       '4.1 (1)', '2.2', '1.2', '3.3 (4)', '4.4 (4)', '1.3', '3.2 (2)',
       '4.2 (3)', '2.3'], dtype=object)

In [46]:
# removing blank spaces
subject_numbers = ["subject-" + number for number in recording_and_metadata_df["Subject"].unique() if number]
# sorting numbers
subject_numbers = sorted(subject_numbers)
subject_numbers_for_title = "_".join(subject_numbers)

In [47]:
subject_numbers_for_title

'subject-1.1_subject-1.2_subject-1.3_subject-1.4_subject-2.1_subject-2.2_subject-2.3_subject-2.4_subject-3.1 (1)_subject-3.2 (2)_subject-3.3 (4)_subject-3.4 (2)_subject-4.1 (1)_subject-4.2 (3)_subject-4.3 (3)_subject-4.4 (4)'

- Getting the experiment name

In [48]:
experiment_names = ["_".join(name.split()) for name in recording_and_metadata_df["Experiment"].unique() if name]
# sorting experiment_names
experiment_names = sorted(experiment_names)
experiment_names_for_title = "AND".join(experiment_names)

In [49]:
experiment_names_for_title

'Pilot_of_Pilot'

- Getting the box numbers

In [50]:
# removing blank spaces
box_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Box"].unique() if number]
# sorting numbers
box_numbers = sorted(box_numbers)
box_numbers_for_title = "_".join(box_numbers)

In [51]:
box_numbers_for_title

'1_2_3_4'

- Getting the script names

In [52]:
# removing blank spaces
script_names = ["_".join(name.split()) for name in recording_and_metadata_df["MSN"].unique() if name]
# sorting names
script_names = sorted(script_names)
script_names_for_title = "_".join(script_names)

In [53]:
script_names_for_title

'levelNP_CS_reward_laserepochON1st_noshock'

## Adding other useful information

- Adding the date as the number of subsequent days since the start

In [54]:
recording_and_metadata_df["date"].unique()

array(['20220516', '20220504', '20220506', '20220507', '20220503',
       '20220510', '20220505', '20220509', '20220508', '20220512',
       '20220511', '20220517'], dtype=object)

In [55]:
for subject in recording_and_metadata_df["subject"].unique():
    print(subject)

2.1
3.1 (1)
1.4
3.4 (2)
4.3 (3)
2.4
1.1
4.1 (1)
2.2
1.2
3.3 (4)
4.4 (4)
1.3
3.2 (2)
4.2 (3)
2.3


In [56]:
subject_to_date_to_session = defaultdict(dict)

for subject in recording_and_metadata_df["subject"].unique():
    subject_df = recording_and_metadata_df[recording_and_metadata_df["subject"] == subject]
    for index, date in enumerate(sorted(subject_df["date"].unique())):
        subject_to_date_to_session[subject][date] = index + 1

In [57]:
subject_to_date_to_session

defaultdict(dict,
            {'2.1': {'20220503': 1,
              '20220504': 2,
              '20220505': 3,
              '20220506': 4,
              '20220507': 5,
              '20220508': 6,
              '20220509': 7,
              '20220510': 8,
              '20220511': 9,
              '20220512': 10,
              '20220516': 11,
              '20220517': 12},
             '3.1 (1)': {'20220503': 1,
              '20220504': 2,
              '20220505': 3,
              '20220506': 4,
              '20220507': 5,
              '20220508': 6,
              '20220509': 7,
              '20220510': 8,
              '20220511': 9,
              '20220512': 10,
              '20220517': 11},
             '1.4': {'20220503': 1,
              '20220504': 2,
              '20220505': 3,
              '20220506': 4,
              '20220507': 5,
              '20220508': 6,
              '20220509': 7,
              '20220510': 8,
              '20220511': 9,
              '2022051

In [58]:
recording_and_metadata_df["session"] = recording_and_metadata_df.apply(lambda x: subject_to_date_to_session[x["subject"]][x["date"]], axis=1)

In [59]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,...,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,session
0,3.95,64.0,399.0,0.0,60.01,5.03,1.0,0.0,20220516,2.1,...,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2,11
1,16.56,144.0,399.0,0.0,140.01,16.70,1.0,0.0,20220516,2.1,...,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2,11
2,16.85,234.0,399.0,0.0,230.01,16.88,1.0,0.0,20220516,2.1,...,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2,11
3,16.97,314.0,399.0,0.0,310.01,17.93,1.0,0.0,20220516,2.1,...,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2,11
4,24.95,389.0,399.0,0.0,385.01,25.28,1.0,0.0,20220516,2.1,...,05/16/22,2.1,Pilot of Pilot,Cage 2,3,11:28:27,12:35:20,levelNP_CS_reward_laserepochON1st_noshock,2,11


In [60]:
recording_and_metadata_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,...,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,session
467539,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),...,05/09/22,4.2 (3),Pilot of Pilot,Cage 3,1,12:43:20,13:48:27,levelNP_CS_reward_laserepochON1st_noshock,3,7
467540,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),...,05/09/22,4.2 (3),Pilot of Pilot,Cage 3,1,12:43:20,13:48:27,levelNP_CS_reward_laserepochON1st_noshock,3,7
467541,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),...,05/09/22,4.2 (3),Pilot of Pilot,Cage 3,1,12:43:20,13:48:27,levelNP_CS_reward_laserepochON1st_noshock,3,7
467542,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),...,05/09/22,4.2 (3),Pilot of Pilot,Cage 3,1,12:43:20,13:48:27,levelNP_CS_reward_laserepochON1st_noshock,3,7
467543,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220509,4.2 (3),...,05/09/22,4.2 (3),Pilot of Pilot,Cage 3,1,12:43:20,13:48:27,levelNP_CS_reward_laserepochON1st_noshock,3,7


In [61]:
recording_and_metadata_df.groupby(["subject", "date", "session"]).count().head(n=50)

(P)Portentry  (Q)USdelivery  (R)UStime  \
subject date     session                                           
1.1     20220503 1                  82             11         11   
        20220504 2                 920             41         41   
        20220505 3                 619             40         40   
        20220506 4                 447             41         41   
        20220507 5                 703             40         40   
        20220508 6                 446             39         39   
        20220509 7                 586             40         40   
        20220510 8                1278             80         80   
        20220511 9                 756             40         40   
        20220516 10                746             41         41   
        20220517 11                424             33         33   
1.2     20220503 1                  68             14         14   
        20220504 2                 364             39         39   
        20220505 3                 548             41         41   
        20220506 4                 412             40         40   
        20220507 5                 464             40         40   
        20220508 6                 471             39         39   
        20220509 7                 611             40         40   
        20220510 8                 441             40         40   
        20220511 9                 631             40         40   
        20220512 10                471             40         40   
        20220516 11                634             40         40   
        20220517 12                686             40         40   
1.3     20220503 1                  24             10         10   
        20220504 2                 195             37         37   
        20220505 3                 112             15         15   
        20220506 4                 366             39         39   
        20220507 5                 503             40         40   
        20220508 6                 536             39         39   
        20220509 7                 464             40         40   
        20220510 8                 498             40         40   
        20220511 9                 749             40         40   
        20220512 10                973             40         40   
        20220516 11               1084             40         40   
        20220517 12                753             40         40   
1.4     20220503 1                  68             11         11   
        20220504 2                 378             41         41   
        20220505 3                   5              3          3   
        20220506 4                 331             38         38   
        20220507 5                 370             39         39   
        20220508 6                 618             39         39   
        20220509 7                 353             40         40   
        20220510 8                1078             80         80   
        20220511 9                 566             39         39   
        20220516 10                642             41         41   
        20220517 11                923             40         40   
2.1     20220503 1                 202             15         15   
        20220504 2                 699             40         40   
        20220505 3                 467             38         38   
        20220506 4                 543             40         40   

                          (W)ITIvalues  (S)CSpresentation  (N)Portexit  \
subject date     session                                                 
1.1     20220503 1                2501                 74           82   
        20220504 2                2501                100          920   
        20220505 3                2501                 99          618   
        20220506 4                2501                100          446   
        20220507 5                2501       

## Adding the strain information

In [62]:
cage_to_strain = {"1": "C57", "2": "C57", "3": "CD1", "4": "CD1"}

In [63]:
recording_and_metadata_df["strain"] = recording_and_metadata_df["cage"].map(cage_to_strain)

In [64]:
recording_and_metadata_df["strain"].unique()

array(['C57', 'CD1'], dtype=object)

## Saving the dataframes(spreadsheets to files) with the metadata as part of the name

- Making necessary directories
    - If you want to use any of the other metadata as part of the name, you will have to swap out the variables in the `format()` and change the name of the folder to match your new name. The variable names are the word that's in front of the `=` at the last line of each cell. The `{}` are where the metadata variables will be inserted into the file name. For more information on formatting strings: https://www.w3schools.com/python/ref_string_format.asp
    - You can also just manually rename the files by replacing everything in `""` and removing the `.format()` part
- **NOTE: You may get an error that the file does not exist. If this is the case, it could be the file name is too long(an issue that may happen when using Jupyter Notebooks on Windows)**

In [65]:
output_directory = "./proc/extracted_recording_data_and_metadata/experiment_{}_cage_{}_date_{}_{}".format(experiment_names_for_title, cage_numbers_for_title, earliest_date, latest_date)

In [66]:
output_directory

'./proc/extracted_recording_data_and_metadata/experiment_Pilot_of_Pilot_cage_1_2_3_4_date_20220503_20220517'

In [67]:
os.makedirs(output_directory, exist_ok=True)

In [68]:
metadata_df["cohort"] = cohort
concatted_medpc_df["cohort"] = cohort
recording_and_metadata_df["cohort"] = cohort

In [69]:
metadata_df.to_csv(os.path.join(output_directory, "metadata_{}_cage_{}_date_{}_{}.csv".format(cohort, cage_numbers_for_title, earliest_date, latest_date)))
# metadata_df.to_excel(os.path.join(output_directory, "metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [70]:
concatted_medpc_df.to_csv(os.path.join(output_directory, "MEDPC_recording_{}_cage_{}_date_{}_{}.csv".format(cohort, cage_numbers_for_title, earliest_date, latest_date)))
# concatted_medpc_df.to_excel(os.path.join(output_directory, "MEDPC_recording_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [71]:
recording_and_metadata_df.to_csv(os.path.join(output_directory, "recording_metadata_{}_cage_{}_date_{}_{}.csv".format(cohort, cage_numbers_for_title, earliest_date, latest_date)))
# recording_and_metadata_df.to_excel(os.path.join(output_directory, "recording_and_metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))